In [ ]:
from langchain_ollama.llms import OllamaLLM

llm_model = OllamaLLM(model="gemma3n:e2b")

chat_title = llm_model.invoke(f"What are the different types of diseases of sugarcane plant ? ")

print(chat_title)
#Identify the plant leaf type () from image: 

In [3]:

llm_model = OllamaLLM(model="gemma3n:e2b")


In [5]:
llm_model.invoke(f"What are the different types of diseases of sugarcane plant ? ")


"## Diseases Affecting Sugarcane Plants: A Comprehensive Overview\n\nSugarcane is susceptible to a variety of diseases, which can significantly reduce yields and quality. Here's a breakdown of the major diseases affecting sugarcane, categorized for clarity:\n\n**1.  Fungal Diseases (Most Common):**\n\n*   **Black Rot (Caused by *Pseudomonas savastanoi* pv. *savastanoi*):**\n    *   **Symptoms:**  Black, water-soaked lesions on leaves, stems, and stalks.  Lesions often develop into necrotic (dead) areas.  Can cause significant yield loss.\n    *   **Severity:**  Can be severe, especially in humid conditions.  Can spread rapidly.\n    *   **Management:**  Good sanitation, avoiding planting in infected areas, resistant varieties, and fungicide applications.\n*   **Diseases caused by *Fusarium* species (e.g., *Fusarium oxysporum f. sp. cubense*):**\n    *   **Symptoms:**  Wilting, yellowing, and necrosis of leaves and stems.  Often starts with yellowing of the lower leaves.  Can cause stem

In [6]:
!pip install inference-sdk

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached contourpy-1.3.2-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached matplotlib-3.10.3-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.5-cp312-cp312-win_amd64.whl.metadata (109 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pypar


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="API_KEY"
)

result = CLIENT.infer('download.jpg', model_id="crop-1wlig/5")